In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *


PATH = "data/dogbreeds/"
sz=224
arch=resnext50
bs=28

ModuleNotFoundError: No module named 'fastai'

In [4]:
#Read reads into df
labels_csv = f'{PATH}labels.csv'
import pandas as pd
labels_df = pd.read_csv(labels_csv)
labels_df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [70]:
#create validation set indexes
n = len(labels_df)
val_idx = get_cv_idxs(n)
val_idx = val_idx[:100]

In [71]:
#define architecture
sz=224 #image size
arch=resnext50 #model
bs=28 #batch size

In [72]:
#create data
tfms=tfms_from_model(arch,sz,aug_tfms=transforms_side_on,max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH,"train",'data/dogbreeds/labels.csv',test_name='test',val_idxs=val_idx,suffix='.jpg',tfms=tfms,bs=bs,num_workers=4)

In [79]:
#compile learn model
#turn precompute to true to quickly train model
learn = ConvLearner.pretrained(arch,data,precompute=True)

In [80]:
#fit model
learn.fit(1e-2, 8)
#turn precompute off to use data augmentations
learn.precompute=False

HBox(children=(IntProgress(value=0, description='Epoch', max=8), HTML(value='')))

[ 0.       0.60377  0.23972  0.9308 ]                         
[ 1.       0.36645  0.22747  0.92411]                         
[ 2.       0.31042  0.18487  0.95759]                         
  6%|▌         | 20/362 [00:00<00:08, 39.25it/s, loss=0.305]

Exception in thread Thread-56:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[ 3.       0.26696  0.19566  0.95759]                         
[ 4.       0.19944  0.16566  0.96429]                         
[ 5.       0.15025  0.15371  0.97321]                         
[ 6.       0.14919  0.1752   0.93304]                         
[ 7.       0.12725  0.16417  0.95759]                         



In [81]:
#learn with precompute false
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.20944  0.15581  0.94866]                        
[ 1.       0.25175  0.19643  0.93304]                        
[ 2.       0.21274  0.19469  0.93304]                        
[ 3.       0.26021  0.18167  0.95536]                        
[ 4.       0.20237  0.17341  0.92411]                        
[ 5.       0.17939  0.1768   0.93304]                        
[ 6.       0.15606  0.16941  0.92411]                        



In [ ]:
#unfreeze other layers
#learn.unfreeze()
#lr=np.array([1e-4,1e-3,1e-2])

In [ ]:
#learn unfreeze layers
#learn.fit(lr, 3, cycle_len=1)

In [82]:
learn.save('my_first_dog_breeds')
learn.load('my_first_dog_breeds')

In [83]:
#return test preds
test_preds,_ = learn.TTA(is_test=True)

In [84]:
#get the mean value of the preds TTA does emsombling
preds_avg = np.mean(test_preds, axis=0)

In [85]:
#select column with highest pred
preds = np.argmax(preds_avg, axis=1)
#create array of probabilies, values are log of probabilities
probs = np.exp(preds_avg)

In [58]:
data.test_ds.fnames

['test/de084b830010b6107215fef5d4a75b94.jpg',
 'test/6b423ca7020e70eb05732843c5d2bad1.jpg',
 'test/74aa7e201e0e93e13e87b986a7d31839.jpg',
 'test/a079f72193264bc5685e5d28d7372680.jpg',
 'test/583f7580fa5fec1266331fcf83b76fd6.jpg',
 'test/72cd7496a319c832f670fbb8d349217e.jpg',
 'test/ed9e75a553dbb4712addc1638693dc03.jpg',
 'test/6322fa7edb40af844609b4843a89b5fd.jpg',
 'test/d18de4f66f7d2ae8dffb08ec13b9bbc5.jpg',
 'test/8c567325012bf8d7daeff453ec959312.jpg',
 'test/bbecb2e61f07719abe32be3f36b3fac0.jpg',
 'test/7d3a1066521da895f915af31e8e739df.jpg',
 'test/a0256a9577f2e7c6631dd78556e4bab8.jpg',
 'test/87dafc44ed8ae792ea5c341d7cbb6df4.jpg',
 'test/1f2e9080823a6b093e7a4baa668393d2.jpg',
 'test/c65172e587bf02794e5b40f830bcb2af.jpg',
 'test/4f0b2dbc23fdef5f939144f456a62140.jpg',
 'test/a5049952a1a2ea0473e6520db5e4a4fb.jpg',
 'test/685ac2356ad42163b2141a41972e5c54.jpg',
 'test/bd405393e16688e5282b57d92bdbb97b.jpg',
 'test/cbb4039e5a16f80c02f3d188f98beaf7.jpg',
 'test/18cf06b0c6a59c6891bf6f5a42b

In [90]:
ds=pd.DataFrame(probs)
ds.columns = data.classes

In [66]:
preds

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,1.245441e-08,1.246618e-07,6.677148e-08,1.608060e-08,8.154742e-09,4.485998e-08,3.575946e-10,2.486815e-08,4.667679e-09,9.486959e-11,...,1.677155e-09,6.382136e-10,3.986086e-09,2.336953e-08,1.746717e-06,5.290183e-09,4.729569e-09,3.194036e-08,1.075730e-09,1.095168e-09
1,1.729775e-07,5.132477e-06,2.712192e-07,1.030715e-06,8.825897e-08,3.173426e-08,2.888126e-06,2.205209e-07,1.980817e-08,5.009726e-08,...,1.756667e-06,1.991796e-07,2.673119e-07,6.719898e-08,3.945522e-08,1.724782e-07,1.652196e-05,6.762418e-08,2.442644e-05,1.289221e-07
2,8.665046e-07,5.141798e-07,1.751647e-07,3.833168e-04,3.146025e-06,7.157262e-07,2.923884e-06,4.610392e-07,3.686566e-07,3.800550e-08,...,6.707838e-07,2.201420e-07,1.819508e-07,5.639007e-07,2.098967e-08,1.299452e-06,2.044834e-06,1.216007e-05,8.033969e-02,2.730895e-07
3,2.539069e-05,2.285392e-05,2.852905e-06,4.219011e-06,3.958028e-07,2.586093e-06,1.504650e-05,7.776088e-07,4.542704e-07,1.619108e-06,...,3.823765e-05,3.026307e-05,7.792598e-07,2.812918e-06,2.365048e-05,1.766118e-05,5.419676e-05,2.920852e-06,1.461410e-04,7.064109e-05
4,4.861858e-06,6.820502e-08,3.094914e-09,7.250509e-07,1.321667e-08,1.281019e-08,1.916186e-08,2.734645e-09,1.956179e-09,3.337660e-09,...,3.735508e-07,7.128301e-08,9.286414e-10,5.332915e-09,1.646238e-09,3.497693e-08,5.328291e-08,2.139390e-08,8.127984e-06,3.586941e-08


In [92]:
ds.insert(0,'id',[o[5:-4] for o in data.test_ds.fnames]) #file name contains 5 letters at start we don't want "/test" and also ".jpg" at end we don't want


In [93]:
ds.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,de084b830010b6107215fef5d4a75b94,2.471379e-10,9.350011e-09,4.261792e-08,1.469241e-09,7.412606e-10,3.527222e-09,6.785306e-10,3.133107e-11,3.283484e-09,...,4.072422e-11,1.299355e-10,5.109884e-10,7.706388e-11,2.641507e-08,1.849269e-10,7.328357e-10,1.079710e-08,2.511840e-10,2.616537e-10
1,6b423ca7020e70eb05732843c5d2bad1,4.362329e-06,8.400004e-07,9.889514e-08,1.595935e-06,5.136024e-09,1.261100e-07,2.008187e-07,1.873238e-07,2.581854e-07,...,1.568205e-07,4.959745e-08,9.223673e-07,5.494473e-08,4.314504e-08,2.974540e-08,6.052117e-06,5.760836e-09,2.003104e-05,3.506691e-08
2,74aa7e201e0e93e13e87b986a7d31839,4.616715e-07,1.310864e-07,2.537647e-09,6.456131e-04,2.320582e-07,5.442424e-07,1.360208e-07,7.177456e-08,1.917981e-07,...,4.819670e-07,1.832177e-08,4.330928e-08,5.349587e-07,9.887793e-09,1.131846e-07,1.680568e-07,7.732077e-06,4.283426e-02,9.037196e-08
3,a079f72193264bc5685e5d28d7372680,1.695434e-04,1.322917e-05,2.399312e-06,1.986126e-05,4.983387e-07,3.044363e-06,7.884894e-06,1.868051e-06,5.748233e-06,...,3.096495e-06,3.870652e-06,4.677668e-06,9.609450e-07,4.640452e-06,5.647912e-06,9.648134e-06,6.418785e-06,7.443245e-04,6.432516e-05
4,583f7580fa5fec1266331fcf83b76fd6,1.393171e-05,2.661703e-07,2.499782e-09,3.301937e-06,1.732676e-08,1.360415e-07,1.367150e-08,2.061328e-08,2.260879e-08,...,2.659645e-07,9.859469e-09,1.164429e-08,1.950848e-08,3.867450e-09,1.123095e-08,4.366795e-08,1.473997e-07,2.282578e-05,1.467959e-08


In [94]:
ds.to_csv('sub3.gz',compression='gzip',index=False)